In [ ]:
# test cell for nifti-saving

def npy_resize_nii(input, reference, output):
    ref = nib.load(reference)
    ref_data = ref.get_fdata()
    swap_data = input.swapaxes(0, 2) 
    input_data = resize(swap_data, ref_data.shape)
    input_data = np.where(input_data>0, 1, 0)
    nii_img = nib.nifti1.Nifti1Image(input_data, affine=ref.affine, header=ref.header)
    nib.save(nii_img, output)

def npy_bbox_nii(input, reference, output, bbox, isMask=False):
    ref = nib.load(reference)
    ref_data = ref.get_fdata() 
    input = input.swapaxes(0,2)

    ref_dim = ref.header['dim']
    new_dim = np.copy(ref_dim)
    new_dim[1] = input.shape[0]
    new_dim[2] = input.shape[1]
    new_dim[3] = input.shape[2]
    ref.header['dim'] = new_dim
    
    print( bbox )
    ref.header['qoffset_x'] = 0
    ref.header['qoffset_y'] = 0
    ref.header['qoffset_z'] = 0
    ref.header['srow_x'] = [1, 0, 0, 0]
    ref.header['srow_x'] = [0, 1, 0, 0]
    ref.header['srow_x'] = [0, 0, 1, 0]

    if isMask:
        input_data = np.where(input_data>0, 1, 0)
    nii_img = nib.nifti1.Nifti1Image(input, affine=ref.affine, header=ref.header)
    nib.save(nii_img, output)

ref_path = r'../testdata/sub-001/ses-01/sub-001_ses-01_ref.nii'
npy_bbox_nii(mask_l, ref_path, 'test.nii', bbox3(pred_l))



In [ ]:
ref_path = r'../testdata/sub-001/ses-01/sub-001_ses-01_ref.nii'

ref_img = nib.load(ref_path)
print(ref_img.header)

ref_dim = ref_img.header['dim']
new_dim = np.copy(ref_dim)
new_dim[1] = mask_l.shape[0]
new_dim[2] = mask_l.shape[1]
new_dim[3] = mask_l.shape[2]
ref_img.header['dim'] = new_dim

bbox = bbox3(pred_l)
ref_img.header['qoffset_x'] -= bbox[0]*ref_img.header['pixdim'][1]
ref_img.header['qoffset_y'] -= bbox[2]*ref_img.header['pixdim'][1]
ref_img.header['qoffset_z'] -= bbox[4]*ref_img.header['pixdim'][1]
ref_img.header['srow_x'][3] = ref_img.header['qoffset_x']
ref_img.header['srow_y'][3] = ref_img.header['qoffset_y']
ref_img.header['srow_z'][3] = ref_img.header['qoffset_z']

print(ref_img.header)

nii_img = nib.nifti1.Nifti1Image(mask_l, affine=ref_img.affine, header=ref_img.header)
nib.save(nii_img, 'test.nii')